In [14]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm, trange

import os
import time
import warnings
warnings.filterwarnings("ignore")
from locanmf import LocaNMF
import postprocess

import scipy

from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax

#

from analysis_fig4 import LocaNMFClass


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means



def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                          13,2,
                          12,3,
                          11,4,
                          10,5,
                          9,6,
                          8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        #print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    #print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/media/cat/4TBSSD/yuki/yongxu/atlas_split.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    #print ("  # of ordered_names: ", ordered_names.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random


def plot_locanmf_vs_raw(locaNMF_temporal, raw_means):
# FIg 2B locaNMF vs. raw

    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    raw_means_clip = raw_means[:,:900]
    print ('raw temporal_means: ', raw_means.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,raw_means.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(raw_means_clip.shape[1])/30-30
    fig=plt.figure(figsize=(10,10))
    linewidth=3
    for k in range(raw_means.shape[0]):
        ax1=plt.subplot(121)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.ylim(-.1,1.25)

        temp1 = raw_means_clip[k]#/np.max(raw_means[k])
        if k==0:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth, 
                     label = 'raw')
        else:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth)

        #plt.plot([-15,0], [scale3+k*scale2, scale3+k*scale2],'--',c='black',alpha=.2)


        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k],
                    label='locaNMF')
        else:
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k])

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    #     # 
    #     ax2=plt.subplot(122)
    #     plt.xlim(-15,0)
    #     temp3 = temp2*scale3-temp1*scale1
    #     plt.plot(t, temp3*scale3+k*scale4,'--', c=colors[k])

    ax1.legend()
    if False:
        plt.savefig('/home/cat/fano.png',dpi=300)
        plt.close()
    else:
        plt.show()       
        
#
def variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)[:,:900]
    var = np.array(var)[:,:900]
    #print (means.shape, var.shape)

    return means, var


def plot_variance_locaNMF(var):
    colors = plt.cm.jet(np.linspace(0,1,var.shape[0]))

    scale1 = 1
    scale2 = .005
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    #print ("t: ", t)
    fig=plt.figure(figsize=(10,10))
    for k in range(var.shape[0]):
        ax=plt.subplot(121)
        plt.xlim(-15,0)
        plt.xticks([])
        plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # 
        #plt.plot(t,temp,'--', c=colors[k],
        plt.plot(t,temp, c=colors[k],
                linewidth=linewidth)

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c=colors[k],alpha=.5)

    if False:
        plt.savefig('/home/cat/variance.png',dpi=300)
        plt.close()
    else:
        plt.show()
  

def plot_longitudinal_roi_loca(n_trials, saved_names, all_means):
    print ('n trials: ', n_trials)
    colors = plt.cm.viridis(np.linspace(0,1,len(all_means)))
    area_ids = [0,1,6,7,8,9,12,13]
    time= np.arange(all_means[0].shape[1])/30-30.

    #
    ctr=1
    min_trials = 10
    aucs = []
    saved = []
    fig=plt.figure(figsize=(10,6))
    for ctr, area_id in enumerate(area_ids): 
        ax = plt.subplot(2,4,ctr+1)

        # 
        aucs.append([])
        counter = 0
        for t in range(len(all_means)):
            temp = all_means[t][area_id]
            if n_trials[t]<min_trials:
                continue

            if np.max(np.abs(temp))<0.2:
                plt.plot(time, temp, 
                         color=colors[t],
                        alpha=.8)

                auc = np.nansum(np.abs(temp), axis=0)

                aucs[ctr].append([t,auc])
                counter+=1

        print (ctr, 'area_id: ', area_id, counter)
        # 
        #if ctr==5:

        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.title(saved_names[area_id],fontsize=8)

        # cmap = matplotlib.cm.viridis
        #norm = matplotlib.colors.Normalize(vmin=5, vmax=10)

        # cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap,
         #                               orientation='vertical')

    #
    if False:
        plt.savefig('/home/cat/'+str(animal_id)+'_loca_longitudinal.png',dpi=300)
        plt.close()
    else:
        plt.show()
        

def load_longitudinal_data_animal(root_dir, animal_id, session_name):

    # 
    loca = LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    all_means = []
    n_trials = []
    saved_names = []
    all_means_random = []
    sessions_good = []
    for session in tqdm(loca.sessions):

        # load data
        fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                     session + '_locanmf.npz')
        atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

        # 
        if atlas is not None:
            means, var = variance_locaNMF(locaNMF_temporal)
            n_trials.append(locaNMF_temporal.shape[0])
            all_means.append(means)

            means, var = variance_locaNMF(random)
            all_means_random.append(means)

            saved_names = names
            
            sessions_good.append(session)
            
    all_means = np.array(all_means)
    all_means_random = np.array(all_means_random)
            
    return all_means, all_means_random, saved_names, sessions_good




def plot_cumulative_longitudinal_locaNMF_imshow_rois(animal_ids):
    
    for animal_id in animal_ids:
        session_name = 'all'
        root_dir = '/media/cat/4TBSSD/yuki/'

        all_means, all_means_random, saved_names, sessions_good = load_longitudinal_data_animal(root_dir, animal_id, session_name)

        #################
        fig=plt.figure(figsize=(10,10))
        img_all = []
        for k in trange(len(sessions_good)):
            img = np.zeros((128,128),'float32')

            # 
            max_vals = np.max(np.abs(all_means[k]),axis=1)

            # 
            data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/tif_files/'+
                           sessions_good[k]+'/'+sessions_good[k]+'_locanmf.npz',
                          allow_pickle=True)

            # 
            A_reshape = data['A_reshape']
            temp = data['temporal_trial']
            local_means = temp.mean(0)
            #max_vals = np.max(np.abs(all_means[k]),axis=1)
            max_vals = np.max(np.abs(local_means),axis=1)

            # 
            for i in range(A_reshape.shape[2]):
                #plt.subplot(4,4,i+1)
                temp = A_reshape[:,:,i]
                idx = np.where(temp!=0)
                #temp[idx]=1
                #idx = np.where(temp<0.75)
                #temp[idx]=0

                # force areas to zero out first
                img[idx]=0
                img+=temp*max_vals[i]

            # 
            img = scipy.ndimage.gaussian_filter(img, sigma=0)

            #
            if k<=100:
                plt.subplot(8,10,k+1)

                plt.imshow(img)
                plt.xticks([])
                plt.yticks([])

            #img_n_smooth = scipy.ndimage.gaussian_filter(img_n, sigma=1)
            img_all.append(img)


        #    plt.title(names[i],fontsize=6)
        plt.tight_layout(h_pad=0.5,w_pad=0.5)
        #plt.suptitle("Animal_id "+animal_id, fontsize=20)
        plt.savefig('/home/cat/'+str(animal_id)+"_allpanels.png", dpi=300)
        plt.close()
        #plt.show()

        fig=plt.figure(figsize=(10,10))
        img_all = np.array(img_all).mean(0)
        plt.imshow(img_all)
        plt.xticks([])
        plt.yticks([])

        #    plt.title(names[i],fontsize=6)
        plt.suptitle("Animal_id "+animal_id, fontsize=20)
        plt.tight_layout(h_pad=0.5,w_pad=0.5)
        plt.savefig('/home/cat/'+str(animal_id)+".png", dpi=300)
        plt.close()

        

In [16]:
# ###########################################################
# ########### LOAD RAW DATA OPTIONAL - FIG 4A ONLY ##########
# ###########################################################

# spatial_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy'
# temporal_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
    
# raw_means = load_raw_data(spatial_fname, temporal_fname)

# Plot locaNMF vs. raw
# plot_locanmf_vs_raw(locaNMF_temporal, raw_means)

In [11]:
# ######################################################
# ###### LOAD LOCANMF DATA AND CORRECT ORDER MAPS ######
# ######################################################

animal_id = 'IJ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

all_means, all_means_random, saved_names, sessions_good = load_longitudinal_data_animal(root_dir, animal_id, session_name)

  7%|▋         | 3/44 [00:00<00:01, 25.41it/s]

# of sessions:  (44,)


100%|██████████| 44/44 [00:02<00:00, 17.99it/s]


In [7]:
############################################################
##### CUMULATIVE + LONGITUDINAL locaNMF ROI PLOTS ##########
############################################################

# # Fig4 supplmenatry 5.3

# animal_ids = ["IA1", "IA2","IA3","IJ1", "IJ2","AQ2"]
# animal_ids = ['IA1']
    
# plot_cumulative_longitudinal_locaNMF_imshow_rois(animal_ids)

  4%|▍         | 3/71 [00:00<00:02, 24.57it/s]

# of sessions:  (71,)


100%|██████████| 52/52 [00:02<00:00, 23.14it/s]


In [13]:
########################################################
######## SIMILAR TO ABOVE BUT USING NORMALIZATION ######
########################################################

# Sup Fig 5.3

# 
fig=plt.figure()
img_all = []
for k in trange(len(sessions_good)):
    img = np.zeros((128,128),'float32')
    
    max_vals = np.max(np.abs(all_means[k]),axis=1)
    #print (max_vals)

    
    data = np.load('/media/cat/4TBSSD/yuki/'+animal_id+'/tif_files/'+
                   sessions_good[k]+'/'+sessions_good[k]+'_locanmf.npz',
                  allow_pickle=True)

    A_reshape = data['A_reshape']
    temp = data['temporal_trial']
    local_means = temp.mean(0)
    #max_vals = np.max(np.abs(all_means[k]),axis=1)
    max_vals = np.max(np.abs(local_means),axis=1)


    for i in range(A_reshape.shape[2]):
        #plt.subplot(4,4,i+1)
        temp = A_reshape[:,:,i]
        #idx = np.where(temp!=0)
        #temp[idx]=1
        idx = np.where(temp<0.0)
        temp[idx]=0
        img+=temp*max_vals[i]
    
    img = scipy.ndimage.gaussian_filter(img, sigma=0)

    #
    if k<=100:
        plt.subplot(8,10,k+1)
    
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
    
    #img_n_smooth = scipy.ndimage.gaussian_filter(img_n, sigma=1)
    img_all.append(img)
    
    
#    plt.title(names[i],fontsize=6)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.suptitle("Animal_id "+animal_id, fontsize=20)
plt.show()

fig=plt.figure()
img_all = np.array(img_all).mean(0)
plt.imshow(img_all)
plt.xticks([])
plt.yticks([])

#    plt.title(names[i],fontsize=6)
plt.suptitle("Animal_id "+animal_id, fontsize=20)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.show()




100%|██████████| 39/39 [00:01<00:00, 19.92it/s]


In [18]:
# ###############################
# ###### VARIANCE ANALYSIS ######
# ###############################

# plot_variance_locaNMF(var)